<a href="https://colab.research.google.com/github/elangbijak4/algoritma-dinda-new/blob/main/k_allay_new1_3_bagus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from collections import defaultdict

# Fungsi untuk menghitung benefit dengan konsep fuzzy
def fuzzy_benefit_function(country, main):
    distance = np.linalg.norm(country['position'] - main['position'])
    power_diff = abs(country['power'] - main['power'])

    # Fuzzy logic: higher benefit for closer distance and smaller power_diff
    benefit = (
        np.exp(-distance) * 0.6 +  # Closer countries get higher weight (60%)
        np.exp(-power_diff) * 0.4  # Smaller power differences get lower weight (40%)
    )
    return benefit, distance, power_diff

# Fungsi untuk memastikan semua negara memiliki klaster
def dynamic_threshold_clustering(countries, main_countries, initial_threshold):
    clusters = defaultdict(list)
    reasons = defaultdict(list)
    threshold = initial_threshold
    unassigned_countries = set(country['id'] for country in countries)

    while unassigned_countries:
        for country in countries:
            if country['id'] not in unassigned_countries:
                continue

            best_main = None
            best_benefit = 0
            best_distance = None
            best_power_diff = None

            for main in main_countries:
                benefit, distance, power_diff = fuzzy_benefit_function(country, main)

                # Fuzzy-based threshold comparison
                if benefit > best_benefit and benefit > threshold:
                    best_main = main
                    best_benefit = benefit
                    best_distance = distance
                    best_power_diff = power_diff

            if best_main is not None:
                clusters[best_main['id']].append(country)
                reasons[best_main['id']].append({
                    'country_id': country['id'],
                    'benefit': best_benefit,
                    'distance': best_distance,
                    'power_diff': best_power_diff
                })
                unassigned_countries.remove(country['id'])

        # Dinamis: Turunkan threshold jika masih ada negara yang belum masuk klaster
        if unassigned_countries:
            threshold *= 0.9  # Turunkan threshold 10% setiap iterasi

    return clusters, reasons

# Data contoh
np.random.seed(42)
countries = [
    {'id': i, 'position': np.random.rand(2) * 100, 'power': np.random.rand() * 100}
    for i in range(50)
]
main_countries = [
    {'id': i, 'position': np.random.rand(2) * 100, 'power': np.random.rand() * 100}
    for i in range(5)
]

# Threshold awal
initial_threshold = 0.5

# Jalankan algoritma
clusters, reasons = dynamic_threshold_clustering(countries, main_countries, initial_threshold)

# Cetak hasil
for main_id, cluster in clusters.items():
    print(f"\nCluster for Main Country {main_id}:")
    for reason in reasons[main_id]:
        print(f"  Country {reason['country_id']} -> Benefit: {reason['benefit']:.4f}, Distance: {reason['distance']:.2f}, Power Diff: {reason['power_diff']:.2f}")


Cluster for Main Country 4:
  Country 18 -> Benefit: 0.3342, Distance: 38.77, Power Diff: 0.18
  Country 22 -> Benefit: 0.0829, Distance: 56.00, Power Diff: 1.57
  Country 25 -> Benefit: 0.0788, Distance: 25.41, Power Diff: 1.62
  Country 10 -> Benefit: 0.0321, Distance: 36.62, Power Diff: 2.52
  Country 27 -> Benefit: 0.0276, Distance: 20.51, Power Diff: 2.67
  Country 9 -> Benefit: 0.0050, Distance: 13.21, Power Diff: 4.38
  Country 32 -> Benefit: 0.0006, Distance: 15.49, Power Diff: 6.49
  Country 42 -> Benefit: 0.0001, Distance: 37.37, Power Diff: 8.33

Cluster for Main Country 3:
  Country 33 -> Benefit: 0.2653, Distance: 70.56, Power Diff: 0.41
  Country 6 -> Benefit: 0.0517, Distance: 30.60, Power Diff: 2.05
  Country 48 -> Benefit: 0.0412, Distance: 55.32, Power Diff: 2.27
  Country 2 -> Benefit: 0.0177, Distance: 83.51, Power Diff: 3.12
  Country 12 -> Benefit: 0.0022, Distance: 50.24, Power Diff: 5.19
  Country 45 -> Benefit: 0.0003, Distance: 43.22, Power Diff: 7.07
  Count